In [ ]:
%pip install --quiet --upgrade diffusers transformers accelerate mediapy peft openai

In [ ]:
import mediapy as media
import random
import sys
import torch

from diffusers import DiffusionPipeline, TCDScheduler
from huggingface_hub import hf_hub_download
num_inference_steps = 12

base_model_id = "stabilityai/stable-diffusion-xl-base-1.0"
repo_name = "ByteDance/Hyper-SD"
plural = "s" if num_inference_steps > 1 else ""
ckpt_name = f"Hyper-SDXL-{num_inference_steps}step{plural}-CFG-lora.safetensors"
device = "cuda"

pipe = DiffusionPipeline.from_pretrained(base_model_id, torch_dtype=torch.float16, variant="fp16").to(device)
pipe.load_lora_weights(hf_hub_download(repo_name, ckpt_name))
pipe.fuse_lora()
pipe.scheduler = TCDScheduler.from_config(pipe.scheduler.config)

In [ ]:

# Generate the Prompt for the Scene
initial_description =  """
Under a moonless sky, Jonathan Harker, a newly qualified English solicitor, arrives at the ancient, towering castle of Count Dracula in the remote Carpathian Mountains. The chill night air is heavy with mist as massive wooden doors swing open. Count Dracula, a tall, gaunt figure with piercing eyes and a sly smile, greets him warmly, yet something unsettling lingers in the air.
"""

In [ ]:
seed = random.randint(0, sys.maxsize)
guidance_scale = 5.0
eta = 0.5

images = pipe(
    prompt = initial_description,
    num_inference_steps = num_inference_steps,
    guidance_scale = guidance_scale,
    eta = eta,
    generator = torch.Generator(device).manual_seed(seed),
    ).images

print(f"Prompt:\t{initial_description}\nSeed:\t{seed}")
media.show_images(images)
images[0].save("output.jpg")

In [ ]:
import requests
import torch
from PIL import Image
from io import BytesIO

from diffusers import StableDiffusionImg2ImgPipeline

device = "cuda"
model_id_or_path = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionImg2ImgPipeline.from_pretrained(model_id_or_path, torch_dtype=torch.float16)
pipe = pipe.to(device)

init_image = Image.open("output.jpg").convert("RGB")
init_image = init_image.resize((768, 512))


prompt = "Unable to resist his growing curiosity, Jonathan Harker decides to explore the castle despite the Count's warnings. Suddenly, three enigmatic women with pale skin and captivating eyes appear, moving toward him with a mesmerizing grace."

images = pipe(
    prompt=prompt,
    image=init_image,
    strength=0.75,
    guidance_scale=7.5
).images

images[1].save("output2.jpg")
media.show_images(images[1])

In [ ]:
prompt = "Just as one of the women reaches out to touch him, a sudden gust of icy wind extinguishes the nearby candles, plunging the corridor into darkness. Jonathan's heart races as he feels an oppressive presence behind him. The candles flare back to life, revealing Count Dracula standing between him and the women, his eyes blazing with a stern warning. "
init_image = Image.open("output.jpg").convert("RGB")

images = pipe(
    prompt=prompt,
    image=init_image,
    strength=0.75,
    guidance_scale=7.5
).images

images[0].save("output3.jpg")
media.show_images(images[2])